# 加载文本数据集

`Ascend` `GPU` `CPU` `数据准备`

[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/mindspore/programming_guide/source_zh_cn/load_dataset_text.ipynb)&emsp;
[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/programming_guide/zh_cn/mindspore_load_dataset_text.ipynb)&emsp;
[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_modelarts.png)](https://authoring-modelarts-cnnorth4.huaweicloud.com/console/lab?share-url-b64=aHR0cHM6Ly9taW5kc3BvcmUtd2Vic2l0ZS5vYnMuY24tbm9ydGgtNC5teWh1YXdlaWNsb3VkLmNvbS9ub3RlYm9vay9tYXN0ZXIvcHJvZ3JhbW1pbmdfZ3VpZGUvemhfY24vbWluZHNwb3JlX2xvYWRfZGF0YXNldF90ZXh0LmlweW5i&imageid=65f636a0-56cf-49df-b941-7d2a07ba8c8c)

## 概述

MindSpore提供的`mindspore.dataset`模块可以帮助用户构建数据集对象，分批次地读取文本数据。同时，在各个数据集类中还内置了数据处理和数据分词算子，使得数据在训练过程中能够像经过pipeline管道的水一样源源不断地流向训练系统，提升数据训练效果。

此外，MindSpore还支持分布式场景数据加载，用户可以在加载数据集时指定分片数目，具体用法参见[数据并行模式加载数据集](https://www.mindspore.cn/docs/programming_guide/zh-CN/master/distributed_training_ascend.html#id6)。

下面，本教程将简要演示如何使用MindSpore加载和处理文本数据。

## 准备环节

1. 准备文本数据，内容如下：

    ```text
    Welcome to Beijing
    北京欢迎您！
    我喜欢English!
    ```

2. 创建`tokenizer.txt`文件并复制文本数据到该文件中，将该文件存放在`./datasets`路径下。执行如下代码完成本步骤。

In [1]:
import os

if not os.path.exists('./datasets'):
    os.mkdir('./datasets')
file_handle=open('./datasets/tokenizer.txt',mode='w')
file_handle.write('Welcome to Beijing \n北京欢迎您！ \n我喜欢English! \n')
file_handle.close()
! tree ./datasets

./datasets
└── tokenizer.txt

0 directories, 1 file


3. 导入`mindspore.dataset`和`mindspore.dataset.text`模块。

In [10]:
import mindspore.dataset as ds
import mindspore.dataset.text as text

## 加载数据集

MindSpore目前支持加载文本领域常用的经典数据集和多种数据存储格式下的数据集，用户也可以通过构建自定义数据集类实现自定义方式的数据加载。各种数据集的详细加载方法，可参考编程指南中[数据集加载](https://www.mindspore.cn/docs/programming_guide/zh-CN/master/dataset_loading.html)章节。

下面演示使用`MindSpore.dataset`模块中的`TextFileDataset`类加载数据集。

1. 配置数据集目录，创建数据集对象。

In [3]:
DATA_FILE = './datasets/tokenizer.txt'
dataset = ds.TextFileDataset(DATA_FILE, shuffle=False)

2. 创建字典迭代器，通过迭代器获取数据，可以获得分词前的数据。

In [4]:
for data in dataset.create_dict_iterator(output_numpy=True):
    print(text.to_str(data['text']))

Welcome to Beijing 
北京欢迎您！ 
我喜欢English! 


## 数据处理

MindSpore目前支持的数据处理算子及其详细使用方法，可参考编程指南中[数据处理](https://www.mindspore.cn/docs/programming_guide/zh-CN/master/pipeline.html)章节。

下面演示构建pipeline，对文本数据集进行混洗和文本替换操作。

1. 对数据集进行混洗。

In [5]:
ds.config.set_seed(58)
dataset = dataset.shuffle(buffer_size=3)

for data in dataset.create_dict_iterator(output_numpy=True):
    print(text.to_str(data['text']))

我喜欢English! 
Welcome to Beijing 
北京欢迎您！ 


2. 对数据集进行文本替换。

In [6]:
replace_op1 = text.RegexReplace("Beijing", "Shanghai")
replace_op2 = text.RegexReplace("北京", "上海")

dataset = dataset.map(operations=replace_op1)
dataset = dataset.map(operations=replace_op2)

for data in dataset.create_dict_iterator(output_numpy=True):
    print(text.to_str(data['text']))

我喜欢English! 
Welcome to Shanghai 
上海欢迎您！ 


## 数据分词

MindSpore目前支持的数据分词算子及其详细使用方法，可参考编程指南中[分词器](https://www.mindspore.cn/docs/programming_guide/zh-CN/master/tokenizer.html)章节。

下面演示使用`WhitespaceTokenizer`分词器来分词，该分词是按照空格来进行分词。

1. 创建`tokenizer`。

In [7]:
tokenizer = text.WhitespaceTokenizer()

2. 执行操作`tokenizer`。

In [8]:
dataset = dataset.map(operations=tokenizer)

3. 创建字典迭代器，通过迭代器获取数据。

In [9]:
for data in dataset.create_dict_iterator(num_epochs=1,output_numpy=True):
    print(text.to_str(data['text']).tolist())

['我喜欢English!']
['Welcome', 'to', 'Shanghai']
['上海欢迎您！']
